# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [1040]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
from cellbell import ding

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [1041]:
%store -r numerosKanoniks
numerosKanoniks=[1,2,3,4,5]
numerosKanoniks=[25]
print numerosKanoniks
%store numerosKanoniks

[25]
Stored 'numerosKanoniks' (list)


In [1042]:
home = expanduser("~")
annee="2020"
annee="20-Kanoniks"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
serie=repertoire+"/"
nomKanoniks=[serie+"K%02d/"%num for num in numerosKanoniks]

In [1043]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [1044]:
PFM.duplicateErrors=[]
for serie in nomKanoniks:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/20-Kanoniks/K25/
ADJ : Genre Nombre
NOM : Genre Nombre
DET : Genre Nombre
VER : Temps Pers Genre
head stems
head stems,VER
head stems,NOM


In [1045]:
getExemple(stems["PREP"])

'sur'

In [1046]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

noir
cuisine.N
sur


In [1047]:
paradigmes.getSigmas(["DET"])

[u'DET, Genre=M, Nombre=Sg',
 u'DET, Genre=M, Nombre=Pl',
 u'DET, Genre=F, Nombre=Sg',
 u'DET, Genre=F, Nombre=Pl',
 u'DET, Genre=N, Nombre=Sg',
 u'DET, Genre=N, Nombre=Pl']

In [1048]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [1049]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')
regles.getRules("VER","Pers=3Pau")

[]

In [1050]:
test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
#test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre","Cas"])
test
#test.index.sortlevel(0,ascending=False)

Nombre,Pl,Sg
Genre,,
F,RAD-u-ra,RAD-u-mi
M,RAD-a-ra,RAD-a-mi
N,RAD-i-ra,RAD-i-mi


In [1051]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [1052]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

sauvage veliS
boisson.F veroma
doux sanuk
viande.M kalo
violet womab
pied.F zepek
acheter.V1 vaseb
chat.F wili
jambon.F tivoSa
talentueux minag
DEM t
rouge Sok
chasser.V1 moziS
chaud dofap
aimer.V2 lidum
patte.N tapana
DEF p
lécher.V2 jogir
boire.V1 divim
fromage.M lebeZe
agripper.V1 laslo
être.V1 kotep
roux Zikir
table.N jolade
avec bib
chocolat.F ljuZu
cuisiner.V2 tasuz
soleil.M Sufawa
joli taman
gâteau.N barneto
Félix.M feliSi
verre.N roSa
manger.V1 natub
fondre.V2 peSir
cabane.M sebi
demander.V2 fewor
sur af
vers wub
de sop
apporter.V2 rapes
dimanche.F deNe
plat.N sugu
repas.F kuse
grand kapeS
sauter.V2 salur
patate.M gakeku
vallée.F kalu
croquette.M ovina
cuisine.N kupa
vieux rekod
journée.N rapum
noir pusis
coquelicot.N vabo
accompagner.V1 poZaw
poulet.F pitamo
préparer.V1 kroSil
offrir.V1 vodas
recouvrir.V2 vaniv
à nak
louche.N biseki
devant lab
enfant.F molo
dans tep
Martin.M martiN
sentir.V2 guSad
pour diS
sous kuk
Johana.F joana
derrière raj
bon pimef
finir.V1 bibel
IND s
hab

In [1053]:
ding()